# Demo Lakehouse

El presente demo muestra a nivel conceptual como funciona una arquitectura Bronze-Silver-Gold.

En el demo, las capas están divididas de la siguiente forma:

* **Bronze**  : ingesta cruda, en este caso es de archivos CSV - normalmente vienen de un sistema externo
* **Silver**  : transformaciones de datos
* **Gold**    : clasificación + agregaciones pensando en las vistas que utilizarían los usuarios finales.

Dataset a utilizar: *winequality-red.csv*

# Capa Gold

En este punto tenemos los datos limpios, con las transformaciones necesarias. Ahora hacemos optimizaciones para poder ejecutar consultas específicas para cumplir con las necesidades de los usuarios

Transformaciones:
* acidity_level derivado de total_acidity
* Agregado por wine_type y acidity_level

Vamos a utilizar la syntaxis de SQL de CTAS para esto.

In [0]:
%sql
USE CATALOG big_data_ii_2025;
USE SCHEMA spark_examples;

CREATE OR REPLACE TABLE wine_gold
USING DELTA AS
WITH classified AS (
  SELECT
      *,
      CASE
        WHEN total_acidity <  8  THEN 'Low'
        WHEN total_acidity < 12 THEN 'Medium'
        ELSE 'High'
      END AS acidity_level
  FROM   wine_silver
)
SELECT
    wine_type,
    acidity_level,
    ROUND(AVG(alcohol),2)   AS avg_alcohol,
    COUNT(*)                AS cnt_samples,
    ROUND(AVG(quality),2)   AS avg_quality
FROM   classified
GROUP  BY wine_type, acidity_level
ORDER  BY wine_type, acidity_level;


Ahora podemos revisar el resultado, la tabla final sobre la cual se harían las consultas

In [0]:
from pyspark.sql.functions import col, lit
df_gold = spark.table("big_data_ii_2025.spark_examples.wine_gold")
df_gold.printSchema()

Y revisar los datos que contiene:

In [0]:
df_gold.display()

Y podemos ver como esto se podria utilizar para contestar preguntas de usuarios finales que no les interesa los detalles del negocio

¿Qué nivel de acidez muestra la mayor calidad promedio en los vinos tintos?

In [0]:
%sql
USE CATALOG big_data_ii_2025;
USE SCHEMA spark_examples;

SELECT
    acidity_level,
    ROUND(avg_quality, 2)     AS avg_quality
FROM   wine_gold   
WHERE  wine_type = 'red'
ORDER  BY avg_quality DESC; 

¿Qué porcentaje del total de muestras corresponde a vinos de acidez “High”?

In [0]:
from pyspark.sql.functions import col, lit, sum, when
agg_df = (
    df_gold
    .agg(
        sum("cnt_samples").alias("total_wines"),
        sum(
            when(col("acidity_level") == "High", col("cnt_samples"))
             .otherwise(0)
        ).alias("high_wines")
    )
)

result_df = agg_df.select(
    (col("high_wines") / col("total_wines") * 100).alias("pct_high_wines")
)
result_df.display()